# Training a CNN classifier with VAE augmented data

### Imports

In [ ]:
import mlflow
import torch
import vae
from utils.mlflow import backend_stores
from utils.trainer import TrainingArguments
from vae.generation import Generator, augmentations
from vae.models import VAEConfig
from evaluation.models import CNNMNIST
from evaluation.train_utils import train_model
from utils.data import load_datasets

vae.models.base.model_store = "pretrained_models/MNIST"

### Seeding, loading data & setting up mlflow logging

In [ ]:
SEED = 1337
DATASET = "MNIST"
# set the backend store uri of mlflow
mlflow.set_tracking_uri(getattr(backend_stores, DATASET))
# seed torch
torch.manual_seed(SEED)
# load datasets
train_dataset, _, val_dataset, test_dataset = load_datasets(DATASET)

### The parameters for the classification task

In [ ]:
training_args = TrainingArguments(
    total_steps=5000,
    batch_size=32,
    validation_intervall=200,
    save_model=False,
    seed=SEED,
    early_stopping=False,
    early_stopping_window=20,
    save_best_metric="best_acc",
)

### Parameters for data augmentation

In [ ]:
MULTI_VAE = False
VAE_EPOCHS = 50
Z_DIM = 2
BETA = 1.0
AUGMENTATION = augmentations.REPARAMETRIZATION
augmentation_params = {"K": 450}

## Training the CNN

In [ ]:
# create a vae config
vae_config = VAEConfig(z_dim=Z_DIM, beta=BETA)
# start mlflow run in experiment
mlflow.set_experiment(f"CNN Z_DIM {Z_DIM}" if AUGMENTATION else "CNN Baseline")
with mlflow.start_run(run_name=AUGMENTATION or "baseline"):
    # perform data augmentation if specified
    if AUGMENTATION is not None:
        gen = Generator(
            vae_config,
            VAE_EPOCHS,
            MULTI_VAE,
            SEED,
        )
        train_dataset = gen.augment_dataset(train_dataset, AUGMENTATION, **augmentation_params)

    # train cnn
    results = train_model(
        model=CNNMNIST(),
        training_args=training_args,
        train_dataset=train_dataset,
        dev_dataset=val_dataset,
        test_dataset=test_dataset,
        seed=SEED,
    )
    # print the results
    print(results)